In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

/Users/mikkeldollerup/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/mikkeldollerup/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/var/folders/ys/szn3cjzn7zg_mdp21slh2_hr0000gn/T/ipykernel_19026/1210242844.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us fe

In [2]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [3]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [4]:
#Scale for better intepretation
data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [21]:
#Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)
data_2023 = data[data['Year'] == 2023]

# OLS

In [23]:
OLS = sm.ols('np.log(Market_share) ~ Price + Range + HP + Chargetime + China', data_2023).fit(cov_type='HC3')
OLS.summary()#.tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     np.log(Market_share)   R-squared:                       0.370
Model:                              OLS   Adj. R-squared:                  0.336
Method:                   Least Squares   F-statistic:                     12.44
Date:                  Mon, 13 May 2024   Prob (F-statistic):           2.84e-09
Time:                          13:13:23   Log-Likelihood:                -195.32
No. Observations:                   100   AIC:                             402.6
Df Residuals:                        94   BIC:                             418.3
Df Model:                             5                                         
Covariance Type:                    HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.9551      1.310     -6.075      0.000     -10.522      -5.389
Price         -0.0288      0.007     -3.950      0.000      -0.043      -0.015
Range          0.1232      0.024      5.133      0.000       0.076       0.170
HP            -0.0149      0.021     -0.725      0.468      -0.055       0.025
Chargetime    -0.0467      0.024     -1.960      0.050      -0.093   -2.04e-06
China         -1.3940      0.583     -2.390      0.017      -2.537      -0.251
==============================================================================
Omnibus:                        7.295   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.026   Jarque-Bera (JB):                7.162
Skew:                          -0.652   Prob(JB):                       0.0279
Kurtosis:                       3.134   Cond. No.                         527.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Willingness to pay

In [24]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

W2P: Range 4.273980184500529
W2P: HP -0.5162070709702602
W2P: Chargetime -1.620686161743024
W2P: China -48.35523902604892


/var/folders/ys/szn3cjzn7zg_mdp21slh2_hr0000gn/T/ipykernel_19026/608251932.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = OLS.params[1] # Price coefficient


NOTE: English delimiter\
25,926 DKK for every 10 increase in Range\
515 DKK for every 10 increase in HP\
-4,093 DKK for every 10 increase in Chargetime\
-691,420 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Logit

In [25]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [26]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.264679,0.668016,1.167516,0.201457,0.236795,0.473476,0.135134,0.121517,2.299359,...,0.18723,0.094104,0.079528,0.036141,0.375668,1.817138,0.180967,0.110838,0.207266,2.362646
U6,0.790714,1.0,0.52821,0.923172,0.159295,0.187237,0.374384,0.106852,0.096085,1.818136,...,0.148045,0.07441,0.062884,0.028578,0.297046,1.436837,0.143093,0.087641,0.163888,1.868178
e-tron,1.496971,1.893188,1.0,1.747737,0.301576,0.354475,0.70878,0.202291,0.181907,3.442073,...,0.280278,0.140871,0.119052,0.054103,0.562364,2.720203,0.270902,0.165922,0.310271,3.536812
e-tron GT,0.856519,1.083222,0.572168,1.0,0.172552,0.202819,0.405541,0.115744,0.104081,1.969445,...,0.160366,0.080602,0.068118,0.030956,0.321767,1.556414,0.155002,0.094935,0.177527,2.023652
Q4 e-tron,4.963831,6.277654,3.315917,5.795352,1.0,1.175409,2.350255,0.67078,0.603188,11.413628,...,0.929378,0.467118,0.394765,0.1794,1.864751,9.019966,0.89829,0.550182,1.028832,11.727775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.550316,0.695973,0.36762,0.642503,0.110865,0.130312,0.260561,0.074366,0.066872,1.265374,...,0.103036,0.051787,0.043766,0.019889,0.206736,1.0,0.099589,0.060996,0.114062,1.300202
C40,5.525869,6.988451,3.691367,6.45154,1.113227,1.308497,2.616366,0.74673,0.671485,12.705955,...,1.034608,0.520008,0.439463,0.199713,2.075891,10.041267,1.0,0.612478,1.145323,13.055671
EX30,9.022156,11.410133,6.026942,10.533512,1.817579,2.136399,4.271775,1.219196,1.096341,20.745175,...,1.689217,0.849024,0.717517,0.326074,3.389334,16.394504,1.632713,1.0,1.869983,21.316161


In [27]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marginal_effects.iloc[i,j] = coefficients[j]*ccp[i]*(1-ccp[i]) #dv/dz*P_i*(1-P_i)


,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-1.174178,-0.004255,0.018186,-0.002196,-0.006896,-0.205753
Leaf_2013,-0.623212,-0.002258,0.009652,-0.001166,-0.00366,-0.109206
Zoe_2013,-0.681985,-0.002471,0.010563,-0.001276,-0.004005,-0.119505
Fortwo_2013,-0.032263,-0.000117,0.0005,-0.00006,-0.000189,-0.005654
Model S_2013,-1.914328,-0.006937,0.029649,-0.003581,-0.011243,-0.33545
...,...,...,...,...,...,...
up!_2023,-0.009926,-0.000036,0.000154,-0.000019,-0.000058,-0.001739
C40_2023,-0.098542,-0.000357,0.001526,-0.000184,-0.000579,-0.017268
EX30_2023,-0.159597,-0.000578,0.002472,-0.000299,-0.000937,-0.027966


In [28]:
marginal_effects[OLS.params.index].mean()

Intercept    -0.213695
Price        -0.000774
Range          0.00331
HP             -0.0004
Chargetime   -0.001255
China        -0.037446
dtype: object

In [29]:
elasticity = fun.elasticity(logit_data, OLS)
elasticity

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:137: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elasticity.iloc[i,j] = ((coefficients[j])*X.iloc[i,j]*(1 - ccp[i]))


,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-6.523208,-0.590975,2.980464,-0.203784,-0.689606,-0.0
Leaf_2013,-7.273519,-0.668677,3.69504,-0.200011,-1.836877,-0.0
Zoe_2013,-7.20181,-0.45189,4.07132,-0.180526,-2.368628,-0.0
Fortwo_2013,-7.922737,-0.604238,1.558403,-0.118565,-2.79186,-0.0
Model S_2013,-3.207957,-0.836499,3.025844,-0.405065,-0.565219,-0.0
...,...,...,...,...,...,...
up!_2023,-7.945194,-0.503438,3.150251,-0.120388,-2.239819,-0.0
C40_2023,-7.855339,-1.225949,5.669588,-0.590722,-1.291784,-0.0
EX30_2023,-7.792198,-1.039836,5.732635,-0.390649,-1.281401,-0.0


In [30]:
elasticity[OLS.params.index].mean()

Intercept    -7.693137
Price        -1.306372
Range          4.67946
HP           -0.359517
Chargetime   -1.547039
China        -0.162447
dtype: object

# Analysis on subsample

### The subsample consists of the 2023 market with:
The 5 highest market share models: Model Y, Model 3, Enyag iV, ID.4 and Q4 e-tron\
The 5 highest market share chinese models: 4, Euniq6, Atto 3, Marvel R and Dolphin\
The highest ccp model: Ocean\
The highest ccp chinese model: Seal\
The highest ccp korean model (so we have models outside EU, US and china): Ioniq 6\
The highest priced model: I7\
Polestar 2 as a random interest


In [31]:
analysis_data = logit_data[logit_data['Year']==2023].copy()
analysis_data = analysis_data[analysis_data['Model'].isin(['I7', 'Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
analysis_data.reset_index(drop=True, inplace=True)
analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year
0,5,2023,0.037958,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,C,DE,2349,1.0,0,0.011269,Q4 e-tron_2023
1,13,2023,0.000679,BMW,I7,60.1,186.519444,44.9,28,Sedan,F,DE,42,1.0,0,0.000987,I7_2023
2,17,2023,0.003587,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,C,CN,222,1.0,1,0.002214,Atto 3_2023
3,18,2023,0.003280,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,C,CN,203,1.0,1,0.003074,Dolphin_2023
4,20,2023,0.000517,BYD,Seal,54.0,32.062000,52.2,38,Sedan,D,CN,32,1.0,1,0.005576,Seal_2023
5,49,2023,0.004686,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,D,DK,290,1.0,0,0.077537,Ocean_2023
6,69,2023,0.003199,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,D,KR,198,1.0,0,0.084790,Ioniq 6_2023
7,100,2023,0.003991,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,M,CN,247,1.0,1,0.000576,Euniq6_2023
8,117,2023,0.011990,MG,4,42.7,26.589191,24.1,41,Hatchback,C,CN,742,1.0,1,0.002143,4_2023
9,119,2023,0.003361,MG,Marvel R,38.8,30.235650,17.7,38,SUV,C,CN,208,1.0,1,0.001509,Marvel R_2023


In [34]:
cross_elasticity = fun.cross_elasticity_1(analysis_data, OLS)
cross_elasticity[:50]

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:195: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]
/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:195: PerformanceWarning: indexing past lexsort depth may impact performance.
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]


Cross_Elasticity
Model_year     Model_year                                
Q4 e-tron_2023 Q4 e-tron_2023 Intercept          0.089644
                              Price              0.021393
                              Range             -0.068866
                              HP                 0.004712
                              Chargetime         0.014742
                              China                   0.0
               I7_2023        Intercept          0.007854
                              Price              0.005309
                              Range             -0.007311
                              HP                  0.00066
                              Chargetime         0.001292
                              China                   0.0
               Atto 3_2023    Intercept          0.017616
                              Price              0.001979
                              Range             -0.011459
                              HP                 0.000662
                              Chargetime         0.003828
                              China              0.003087
               Dolphin_2023   Intercept          0.024457
                              Price              0.002067
                              Range             -0.016175
                              HP                 0.000425
                              Chargetime         0.005746
                              China              0.004286
               Seal_2023      Intercept           0.04436
                              Price              0.005154
                              Range             -0.037101
                              HP                 0.004332
                              Chargetime           0.0099
                              China              0.007773
               Ocean_2023     Intercept          0.616821
                              Price              0.134647
                              Range             -0.668741
                              HP                 0.064962
                              Chargetime         0.126793
                              China                   0.0
               Ioniq 6_2023   Intercept          0.674515
                              Price              0.106644
                              Range             -0.592346
                              HP                 0.040377
                              Chargetime         0.063384
                              China                   0.0
               Euniq6_2023    Intercept           0.00458
                              Price               0.00063
                              Range             -0.002511
                              HP                 0.000149
                              Chargetime          0.00121
                              China              0.000803
               4_2023         Intercept          0.017044
                              Price              0.001642

# Cost

In [16]:
logit_data['Cost_OG']=fun.cost_original(logit_data, alpha)
logit_data['Cost']=fun.cost(logit_data, alpha)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost_OG,Cost
0,10,2013,0.002237,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.144783,I3_2013,20.505806,-11.295887
1,132,2013,0.472036,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.135095,Leaf_2013,21.175532,0.000000
2,158,2013,0.205817,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.200516,Zoe_2013,11.090824,0.000000
3,167,2013,0.002237,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.022899,Fortwo_2013,20.334942,0.000000
4,173,2013,0.250559,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.408012,Model S_2013,59.291257,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.001277,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.004881,up!_2023,17.333791,0.000000
330,188,2023,0.008354,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.016984,C40_2023,42.539280,0.000000
331,189,2023,0.000065,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.021896,EX30_2023,36.144829,0.000000
332,190,2023,0.031752,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.015324,XC40_2023,43.450994,0.000000


In [17]:
logit_data[logit_data['Cost']>logit_data['Price']]

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost_OG,Cost


In [18]:
logit_data[logit_data['Cost_OG']>logit_data['Price']].shape[0]

0

In [19]:
logit_data[logit_data['Cost']<0].shape[0]

1

In [20]:
logit_data[logit_data['Cost_OG']<0]

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost_OG,Cost
